# Hackathon Day 2
Today we want to train a Convolutional Neural Network for classifying pneumonia on the [PneumoniaMNIST](https://medmnist.com/) dataset


**Learning Outcome:** Learn how to
- define training and model parameters and analyse their influence on model performance
- write a training loop
- evaluate classification performance using different performance metrics 

# Task 1: Load dataset 
Use the code from day 1 to load your dataset.\
If you worked on this yesterday, you can also use data augmentation methods. Data augmentation is a technique used to increase the diversity of training data by applying random transformations, like rotating, flipping, or cropping images, to create new variations of existing data. This helps improve a model’s ability to generalize and perform better on unseen data.\
For data augmentation you can use the [MONAI library](https://docs.monai.io/en/stable/transforms.html)

In [ ]:
# import some libraries we will need 

from medmnist import PneumoniaMNIST 
import numpy as np
import torch
import random
from matplotlib import pyplot as plt
import torch.nn as nn
from monai.data import DataLoader, CacheDataset
from monai.utils import  GridSamplePadMode
from monai.transforms import (
    Compose,
    EnsureTyped,
    ScaleIntensityd
)

In [ ]:
data_transforms = Compose(
    [
        EnsureTyped(keys=['image'], data_type='tensor'),
        ScaleIntensityd(keys=['image'], minv=0, maxv=1 )
    ] )

# Optional: Apply Data Augmentation:
# You can also randomly rotate or flip your dataset, apply gaussian noise, etc. to create more variability in the dataset 
# see https://docs.monai.io/en/stable/transforms.html
train_transforms = Compose(
    [
        ...        
    ])
    
# If you want to use Monai data transformations you have to use this class
# to convert medmnist dataset to a format compatible with MONAI
class WrappedPneumoniaMNIST(PneumoniaMNIST):
    def __getitem__(self, index):
        image, label = super().__getitem__(index)
        image = torch.tensor(np.array(image)).unsqueeze(0)
        return {"image": image, "label": label}

# Load Pneumonia dataset for training, validation and test
size=... # for first experiments you can use 28x28 to get fast results but in the end you should work with a higher resolution, e.g. 224x224
train_dataset = WrappedPneumoniaMNIST(split="train",download=True, size=size)
val_dataset = ...
test_dataset = ...

# Wrap with MONAI CacheDataset
# Attention! If you use transformations to augemnt your dataset, only apply it to the training data and not to the validation and test set, as we want to leave these unchanged.
train_monai_dataset = CacheDataset(data=train_dataset, transform=train_transforms, cache_rate=1.0, num_workers=4)
val_monai_dataset = ...
test_monai_dataset = ...

In [ ]:
# Next: initiliaze train_loader, val_loader and test_loader using DataLoader from MONAI
BATCH_SIZE = ...
train_loader = DataLoader()
val_loader = ...
test_loader = ...

# Task 2: Define training and model parameters
- We want to train a classification model. You can for example use the **ResNet18** model. To initialize the model you can use [Monai](https://docs.monai.io/en/stable/networks.html#nets). 
- Moreover, we have to define our training parameters, as optimizer, loss function and learning rate. 
- For a binary classification problem you can for example use binary cross entropy loss (see [here](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)). 
- As optimizer you can use stochastic gradient descent (see [here](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)) 

Carry out several experiments and vary the model parameters. What effect does this have on the model performance?\
You can use, for example, a larger model (e.g. ResNet 50), and vary the learning rate and the batch size. Write down your observations.  
For quick experiments, you can work with the small data set (28x28). Ultimately, however, you should train a model on one of the higher-resolution data sets (64x64 or 224x224). 

In [ ]:
from monai.networks.nets import resnet18

device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18().to(device) # use .to(device) to move the model to the gpu
criterion = nn.BCEWithLogitsLoss()

NUM_EPOCHS = 100
lr=0.001 
wd=0.001  
# You can use weight decay as a penalty term for the model weights. 
# This penalty term prevents the weights from becoming to large and thus helps the model not only to memorize the training data, 
# but also to learn general patterns that can be applied to new data (prevents overfitting)

optimizer = ...


# Task 3: Evaluation function 
- Use [scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html) to evaluate your model
- For a classification task you can for example use metrics like AUC, Accuracy, F1-Score, Sensitivity and Specificity

What exactly do the individual metrics mean? How can you interpret the results?

In [ ]:
from sklearn.metrics import (
    accuracy_score
    ...
    )

def evalModel(y_true, y_score, thresh=0.5):
    y_true = y_true.squeeze() # ground truth class / label
    y_score = y_score.squeeze() # probabilities 

    y_pred = (y_score > thresh).astype(int) # predicted class

    metric_dict = {}

    metric_dict['Acc'] = accuracy_score(y_true, y_pred)
    # calculate further evaluation metrics as AUC, F1-Score, sensitivity and specificity and save them in the dictionary metric_dict

   
    return metric_dict

# Task 4: Training loop
A training loop is the repetitive process in which a model learns from data by iteratively passing batches through the network, calculating loss, updating weights using an optimizer, and repeating until the model converges or a set number of iterations is reached.

In [ ]:
import os
import time
from tqdm import tqdm

final_model_path = os.path.join(os.path.dirname(os.path.realpath('__file__')),f'finalModel_{NUM_EPOCHS}_Epochs_size_{size}.model')
best_model_path = os.path.join(os.path.dirname(os.path.realpath('__file__')),f'bestModel_{NUM_EPOCHS}_Epochs_size_{size}.model')

# to save performances 
writer_dict = {'epoch': [],
               'loss_train': [],
               'loss_valid': [],
               'lr': [],
               'Acc': [],
               'F1': [],
               'Sensitivity': [],
               'Specificity': [],
               'AUC': []}

train_loss = []
val_loss = []

min_valid_loss = None
scaler = torch.amp.GradScaler() 
# https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html
# Gradient scaling helps prevent gradients with small magnitudes from flushing to zero (“underflowing”) when training with mixed precision.

for epoch in range(NUM_EPOCHS): # iterate over the epochs

    start_time = time.time()
  
    model.train()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        with torch.autocast(device_type='cuda'):
        
            inputs = batch['image'].to(device) # move to gpu
            targets = ... # get labels saved in batch
        
            outputs = ... # calculate the predicted output by giving your model the input images
                    
            loss = criterion(...)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss.append(loss.item())
    
    lr = optimizer.param_groups[0]['lr'] # to get actual learning rate (only important if learning rate is not constant over time)
    writer_dict['lr'].append(lr) 
    
    if epoch == NUM_EPOCHS-1: # check if last epoch is reached
        # You can use this code to save your model
        state_dict = model.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].cpu()

        torch.save({
            'epoch': epoch,
            'save_dir': final_model_path,
            'state_dict': state_dict,
            'optimizer_state_dict': optimizer.state_dict()},
            final_model_path)

    # Now we want to evaluate our model on the validation data
    model.eval()
    y_true = torch.tensor([]).to(device)
    y_score = torch.tensor([]).to(device)
    
    with torch.no_grad():
        for batch in val_loader:
            with torch.autocast(device_type='cuda'):

                inputs = ...
                targets = ...
                outputs = ...
            
            val_loss.append(...)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs.sigmoid()), 0) # sigmoid to transform outputs from logits to probabilites 
    
    y_true = y_true.cpu().numpy() # move from gpu to cpu and convert tensor to numpy array
    y_score = y_score.detach().cpu().numpy()

    
    # Save best model on validation set 
    if epoch == 0:
        min_valid_loss = np.mean(val_loss)

        ... # save model

       
    elif np.mean(val_loss) < min_valid_loss:
        min_valid_loss = np.mean(val_loss)

         ... # save model

    # to monitor model performance  
    writer_dict['epoch'].append(...)
    writer_dict['loss_train'].append(...)
    writer_dict['loss_valid'].append(...)

    # eval your model usign the evaluation function from task 3
    metric_dict_val = ... 

    for cur_key in metric_dict_val.keys():
        writer_dict[cur_key].append(metric_dict_val[cur_key])
    
    end_time = time.time()
 
    print('Epoch %03d, time for epoch: %3.2f' %(epoch, end_time-start_time))
    print('loss %2.4f, validation loss %2.4f' %(...))  

## Task 5: Final model evaluation
- Plot the loss function and the determined evaluation metrics
- Write an evaluation / inference function to evaluate the trained model on the test set
- Check the false-positive and false negative cases

Take a closer look at the results of the model. Are there any reasons why the model was wrong here?

In [ ]:
# Plot training and validation loss saved in writer_dict
from matplotlib import pyplot as plt
plt.figure()
plt.plot(...)


In [ ]:
# plot evaluation metrics saved in writer_dict
plt.figure()
plt.plot(...)

In [ ]:
# save writer_dict 
# see: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict-or-any-other-python-object
import pickle

savePath = os.path.join(os.path.dirname(os.path.realpath('__file__')),f'writer_dict_{NUM_EPOCHS}_Epochs_size_{size}.pickle')

# write dictionary
with open(...) as f:
    ...
# load dictionary
with open(...) as f:
    ...

## Model inference
Now we want to use our trained model to predict new data.\
Therefore, we define a function *infer*, which gets a data loader and the trained model as input. \
You can use the code from the training loop for help.

In [ ]:
def infer(...):
    model.eval()
    y_true = torch.tensor([]).to(device)
    y_score = torch.tensor([]).to(device)
    
    with torch.no_grad():
        ...


        metric_dict = evalModel(...)

        y_pred = (y_score >= 0.5).astype(int)

        print('auc: %.3f  acc:%.3f, f1: %3f, sens.: %3f, spec.: %3f' %(metric_dict['AUC'],metric_dict['Acc'], metric_dict['F1'],metric_dict['Sensitivity'], metric_dict['Specificity']))
        

    return metric_dict, y_pred

In [ ]:
# You can use the code belwo to load the best model
model = resnet18(spatial_dims=2,num_classes=1,n_input_channels=1).to(device)
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['state_dict'])

model.eval()
print('==> Evaluating ...')
print('Validation performance:')
metrics_val, val_pred = infer(...)
print('Test performance')
metrics_test, test_pred = infer(...)

# Now we want to convert our dictionaries to a pandas dataframe and save the results as csv file
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html
import pandas as pd
df_test = pd.DataFrame(...)
df_test.to_csv(...)

df_val = pd.DataFrame(...)
df_val.to_csv(...)

Now we want to have a closer look on our model performance and the predicted results from our test set. \
Write code, to visualize false positive (prediction = Pneumonia, label = Healthy) and false negative cases (prediction = Healthy, label = Pneumonia)

In [ ]:
# Check false positive and false negative test cases
# save indices for fp and fn in a separate list
fp_cases = []
fn_cases = []
for cur_ind in range(len(test_dataset)):
    if test_pred[cur_ind] == 1 and test_dataset.labels[cur_ind]==0:
        ...
    elif test_pred[cur_ind] == 0 and test_dataset.labels[cur_ind]==1:
        ...

# plot fp and fn cases using plt.subplots
...

# Optional Tasks
When you have finished everything else, you can continue working on one of the following tasks, for example, or come up with something of your own
## Task 1: Multi-label classification
So far we have only dealt with a binary classification problem. However, we often also have to deal with multi-label problems. Multi-label classification is a type of classification where each data point can belong to multiple classes simultaneously, meaning the model predicts a set of labels rather than just one. For example, a photo might be labeled as both "cat" and "dog" if it contains both animals.
You can use and adapt the code you have developed here to solve a multi-label classification problem using the ChestMNIST from the [MedMnist](https://medmnist.com/) data set.

## Task 2: Large Language Models
Use Large Language Models (LLMs) to generate radiological reports for the PneumoniaMNIST dataset. See for example  https://huggingface.co/microsoft/Phi-3-vision-128k-instruct

## Task 3: Generative data augmentation
For generative data augmentation you can for example train a conditional Generative Adversarial Network (GAN) to generate new healty and new pneumonia images, see for example: https://github.com/qbxlvnf11/conditional-GAN/blob/main/conditional-GAN-generating-fashion-mnist.ipynb 
GANs are relatively difficult to train, which is why you can also consider using a pre-trained GAN to achieve better results. You can take a look at this: GANs are relatively difficult to train, which is why you can also consider using a pre-trained GAN to achieve better results. You can take a look at this: https://github.com/Project-MONAI/tutorials/blob/main/modules/mednist_GAN_tutorial.ipynb.


